In [1]:
def get_counts(ckt):
  from qiskit import Aer
  ckt_copy = ckt.copy()
  sim = Aer.get_backend('aer_simulator')
  return sim.run(ckt_copy).result().get_counts()
  

In [2]:
def draw_histogram(ckt):
  from qiskit.visualization import plot_histogram
  import matplotlib.pyplot as plt
  plt.style.use('dark_background')
  return plot_histogram(get_counts(ckt))

In [3]:
def list_backends():  
    from qiskit_ibm_runtime import QiskitRuntimeService
    service = QiskitRuntimeService()

    for backend in service.backends():
        qubit_count = backend.num_qubits
        print(f"{backend.name}: {backend.status().pending_jobs} jobs, {qubit_count} qubits")

In [4]:
def draw_statevector(ckt, output="latex"):
  from qiskit.quantum_info import Statevector
  ket = Statevector(ckt)
  return ket.draw(output=output)

In [5]:
def draw_statevector_ket(ckt):
  from qiskit.quantum_info import Statevector
  from qiskit.visualization import array_to_latex
  ket = Statevector(ckt)    
  return array_to_latex(ket, prefix="\\text{Statevector} = ")
  

In [1]:
def draw_unitary(ckt, prefix=""):
    """Simulates a simple circuit and display its matrix representation.
    Args:
        qc (QuantumCircuit): The circuit to compile to a unitary matrix
        prefix (str): Optional LaTeX to be displayed before the matrix
    Returns:
        None (displays matrix as side effect)
    """
    from qiskit import Aer
    from qiskit.visualization import array_to_latex
    sim = Aer.get_backend('aer_simulator')
    # Next, we'll create a copy of the circuit and work on
    # that so we don't change anything as a side effect
    ckt_copy = ckt.copy()
    # Tell the simulator to save the unitary matrix of this circuit
    ckt_copy.save_unitary()
    unitary = sim.run(ckt_copy).result().get_unitary()
    return display(array_to_latex(unitary, prefix="U="))

In [ ]:
def draw_qsphere(ckt):
  from qiskit import Aer, assemble
  from qiskit.visualization import plot_state_qsphere
  sim = Aer.get_backend('aer_simulator')
  ckt_copy = ckt.copy()
  ckt_copy.save_statevector()
  qobj = assemble(ckt_copy)
  result = sim.run(qobj).result()
  state = result.get_statevector()
  return plot_state_qsphere(state)

In [ ]:
def q_plot_histogram(ckt, backend="ibmq_qasm_simulator"):
  from qiskit_ibm_runtime import QiskitRuntimeService, Session, Options, Sampler
  from qiskit.visualization import plot_histogram
  import matplotlib.pyplot as plt

  service = QiskitRuntimeService()
  options = Options(optimization_level=1)
  options.execution.shots = 1024

  with Session(service=service, backend=backend) as session:
      sampler = Sampler(session=session, options=options)
      job = sampler.run(circuits=ckt)
      result = job.result()
      print(f"Backend: {backend}")
      print(f"Job ID is {job.job_id()}")
      print(f"Job status is {job.status()}")
      print(f"Job result is {result}")
      plt.style.use('dark_background')
      distribution = result.quasi_dists[0].binary_probabilities()
      return plot_histogram(distribution)